### A pyiron workflow to calculate a grain boundary energy using multiple potentials, and compare to literature DFT data.

This notebook __doesn't__ work out of the box if you haven't configured LAMMPS for pyiron

conda install -c conda-forge lammps 

OR

mamba install -c conda-forge lammps

In the python environment in which you are running pyiron

WARNING: In practice/production you should configure a compiled LAMMPS executable that is optimised for your cluster!

In [1]:
from pyiron.project import Project

import numpy as np
import pandas as pd

from ase.lattice.cubic import BodyCenteredCubic as bcc
from ase.build import stack

from pymatgen.core import Structure
from pymatgen.io.ase import AseAtomsAdaptor

from pyiron.atomistics.structure.atoms import ase_to_pyiron, pymatgen_to_pyiron

In [2]:
def add_vacuum(structure, vacuum):
    """
    Adds padding to a slab or 2D material.

    Args:
        structure (Structure): Structure to add vacuum to
        vacuum (float): Vacuum thickness to add in Angstroms
    Returns:
        Structure object with vacuum added.
    """
    struct = structure.copy()
    coords = [s.coords for s in struct.sites]
    species = [s.specie for s in struct.sites]
    lattice = struct.lattice.matrix
    lattice.setflags(write=1)
    lattice_C = lattice
    lattice_C[2][2] += vacuum
    struct = Structure(lattice_C, species, coords, coords_are_cartesian=True)
    struct.translate_sites(range(len(struct)), [0,0,vacuum/2], frac_coords=False)
    return struct

In [3]:
pr = Project("GB_calcs")

Create the structure that we're going to use:
a $\Sigma 3 [1 \bar{1}0](111)$ Fe GB

In [4]:
surface1 = [1, 1, 1]
surface2 = [1, 1, -1]
rotation_axis = [1, -1, 0]
element = "Fe"
lc = 2.8318488966083
GB_name = "S3-RA110-S1-11"

# The minimum required length of the cell.
req_length = 30

v1 = list(-np.cross(rotation_axis,surface1))
v2 = list(-np.cross(rotation_axis,surface2)) 

length = 0
n = 0

vacuum_slab = 15
while length < req_length:
    n += 1
    
    slab1 = bcc(symbol=element, latticeconstant=lc,directions=[rotation_axis,v1,surface1], size=[1,1,n])
    slab2 = bcc(symbol=element, latticeconstant=lc,directions=[rotation_axis,v2,surface2], size=[1,1,n])

    gb = stack(slab1, slab2)
    slab = stack(slab1, slab1)
    length = gb.cell[-1,-1]

    # Rattle the gb structure using rattle in ASE to perturb symmetry
    gb.rattle()
    slab.rattle()
        
    slab_pmg = add_vacuum(AseAtomsAdaptor().get_structure(slab).copy(), vacuum_slab)
    gb_pmg = add_vacuum(AseAtomsAdaptor().get_structure(gb).copy(), vacuum_slab)

Create the pyiron job, assign the structure to the job.

In [5]:
job = pr.create_job(job_type=pr.job_type.Lammps, job_name='Fe_S3_GB')

job.structure = ase_to_pyiron(gb)

We need to see the potentials, pyiron has an inbuilt potential database

It automatically finds the applicable potentials given the structure provided.

In [6]:
job.list_potentials()[:10]

['1997--Ackland-G-J--Fe--LAMMPS--ipr1',
 '1998--Meyer-R--Fe--LAMMPS--ipr1',
 '2001--Lee-B-J--Fe--LAMMPS--ipr1',
 '2001--Lee-B-J--Fe-Cr--LAMMPS--ipr1',
 '2003--Mendelev-M-I--Fe-2--LAMMPS--ipr3',
 '2003--Mendelev-M-I--Fe-5--LAMMPS--ipr1',
 '2004--Ackland-G-J--Fe-P--LAMMPS--ipr1',
 '2004--Zhou-X-W--Fe--LAMMPS--ipr2',
 '2005--Lee-B-J--Fe-Cu--LAMMPS--ipr1',
 '2005--Mendelev-M-I--Al-Fe--LAMMPS--ipr1']

In [7]:
def run_lammps_calc(structure, job_name, potential):
    job = pr.create_job(job_type=pr.job_type.Lammps, job_name=job_name)
    job.structure = structure
    job.potential = potential
    job.calc_minimize(pressure=None,
                        max_iter=10000)
    job.run(delete_existing_job=True)
    return job['output/generic/energy_tot'][-1]

In [8]:
potentials_tested = job.list_potentials()
GB_energy_list = []
for i, potential in enumerate(potentials_tested):
    try:
        job_GB_name = f"S3_GB_{i}"
        GB_toten = run_lammps_calc(pymatgen_to_pyiron(gb_pmg), job_GB_name, potential)
        
        job_SLAB_name = f"S3_SLAB_{i}"
        SLAB_toten = run_lammps_calc(pymatgen_to_pyiron(slab_pmg), job_SLAB_name, potential)
        
        # Formula for GB energy is E_GB_tot - E_SLAB_tot / 2 * (Interface area)
        # The 2 in the denominator is to account for the second interface present in the cell due to the periodic conditions
        
        GB_energy = (GB_toten - SLAB_toten) \
                    / (gb_pmg.volume / gb_pmg.lattice.c) * 16.02
        
        GB_energy_list.append(GB_energy)
    except:
        GB_energy_list.append(np.nan)

The job S3_GB_0 was saved and received the ID: 2507
The job S3_SLAB_0 was saved and received the ID: 2508
The job S3_GB_1 was saved and received the ID: 2509


2023-08-17 00:07:00,837 - pyiron_log - WARNING - Job aborted - please remove it and run again! S3_SLAB_1


The job S3_SLAB_1 was saved and received the ID: 2511
The job S3_GB_2 was saved and received the ID: 2512
The job S3_SLAB_2 was saved and received the ID: 2515
The job S3_GB_3 was saved and received the ID: 2517
The job S3_SLAB_3 was saved and received the ID: 2518
The job S3_GB_4 was saved and received the ID: 2520
The job S3_SLAB_4 was saved and received the ID: 2523
The job S3_GB_5 was saved and received the ID: 2525
The job S3_SLAB_5 was saved and received the ID: 2527
The job S3_GB_6 was saved and received the ID: 2529
The job S3_SLAB_6 was saved and received the ID: 2531
The job S3_GB_7 was saved and received the ID: 2532
The job S3_SLAB_7 was saved and received the ID: 2534
The job S3_GB_8 was saved and received the ID: 2535
The job S3_SLAB_8 was saved and received the ID: 2536
The job S3_GB_9 was saved and received the ID: 2538
The job S3_SLAB_9 was saved and received the ID: 2540
The job S3_GB_10 was saved and received the ID: 2541
The job S3_SLAB_10 was saved and received the

2023-08-17 00:07:17,765 - pyiron_log - WARNING - Job aborted - please remove it and run again! S3_GB_15


The job S3_GB_15 was saved and received the ID: 2557


...

The job S3_GB_124 was saved and received the ID: 2822


2023-08-17 00:09:04,307 - pyiron_log - WARNING - Job aborted
2023-08-17 00:09:04,307 - pyiron_log - WARNING - LAMMPS (28 Mar 2023 - Update 1)
Reading data file ...
  orthogonal box = (0 0 0) to (4.0048391 6.9365848 49.334343)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  84 atoms
  read_data CPU = 0.000 seconds
ERROR: KIM ModelCreate failed (src/KIM/pair_kim.cpp:828)
Last command: pair_style kim Sim_LAMMPS_TersoffZBL_ByggmastarGranberg_2020_Fe__SM_958863895234_000

2023-08-17 00:09:04,321 - pyiron_log - WARNING - Job aborted - please remove it and run again! S3_GB_126


The job S3_GB_125 was saved and received the ID: 2823


2023-08-17 00:09:04,553 - pyiron_log - WARNING - Job aborted
2023-08-17 00:09:04,553 - pyiron_log - WARNING - LAMMPS (28 Mar 2023 - Update 1)
Reading data file ...
  orthogonal box = (0 0 0) to (4.0048391 6.9365848 49.334343)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  84 atoms
  read_data CPU = 0.000 seconds
ERROR: KIM ModelCreate failed (src/KIM/pair_kim.cpp:828)
Last command: pair_style kim Sim_LAMMPS_TersoffZBL_HenrikssonBjorkasNordlund_2013_FeC__SM_473463498269_000



The job S3_GB_126 was saved and received the ID: 2824


Construct a dataframe which contains the information about GB energy, and their errors with respect to a DFT computed value.

The GB energy of 1.58 J/m^2 is from the value computed in this study:

Mai, H.L., Cui, X.Y., Scheiber, D., Romaner, L. and Ringer, S.P., 2022. The segregation of transition metals to iron grain boundaries and their effects on cohesion. Acta materialia, 231, p.117902.

In [9]:
data = {'potential': potentials_tested, 'GB_energy': GB_energy_list}
df = pd.DataFrame(data)

GB_energy_DFT = 1.58
df["err_DFT"] = np.round(GB_energy_DFT - df["GB_energy"], 3)
df["abs_rel_err_DFT"] = abs(np.round((GB_energy_DFT - df["GB_energy"]) / GB_energy_DFT * 100, 0))
df["GB_energy"] = np.round(df.GB_energy.tolist(), 3)
df = df.dropna(subset="GB_energy")
df

,potential,GB_energy,err_DFT,abs_rel_err_DFT
0,1997--Ackland-G-J--Fe--LAMMPS--ipr1,1.547,0.033,2.0
1,1998--Meyer-R--Fe--LAMMPS--ipr1,1.324,0.256,16.0
2,2001--Lee-B-J--Fe--LAMMPS--ipr1,1.221,0.359,23.0
3,2001--Lee-B-J--Fe-Cr--LAMMPS--ipr1,1.185,0.395,25.0
4,2003--Mendelev-M-I--Fe-2--LAMMPS--ipr3,1.296,0.284,18.0
...,...,...,...,...
63,2021--Starikov-S--Fe--LAMMPS--ipr2,5.006,-3.426,217.0
64,2021--Wen-M--Fe-H--LAMMPS--ipr1,1.309,0.271,17.0
65,2022--Mahata-A--Al-Fe--LAMMPS--ipr1,1.130,0.450,29.0
66,2022--Starikov-S--Fe-Cr-H--LAMMPS--ipr1,1.487,0.093,6.0


Order the potentials by the minimum error wrt. DFT!

Now we have a good idea of how to search for an empirical potential which best captures the GB energetics of a specific grain boundary.

You can see that there are some potentials that are very, very accurate at predicting this specific GB energy. 

For a real study you should consider not just a single GB, but validate it across multiple GBs and ideally the specific phenomena you want to study.

In [10]:
df.sort_values(by="abs_rel_err_DFT").head(10)

,potential,GB_energy,err_DFT,abs_rel_err_DFT
0,1997--Ackland-G-J--Fe--LAMMPS--ipr1,1.547,0.033,2.0
44,2017--Beland-L-K--Fe-Ni-Cr--LAMMPS--ipr1,1.632,-0.052,3.0
31,2011--Bonny-G--Fe-Ni-Cr--LAMMPS--ipr2,1.641,-0.061,4.0
66,2022--Starikov-S--Fe-Cr-H--LAMMPS--ipr1,1.487,0.093,6.0
32,2011--Chiesa-S--Fe-33--LAMMPS--ipr1,1.671,-0.091,6.0
23,2009--Olsson-P-A-T--Fe--LAMMPS--ipr1,1.468,0.112,7.0
7,2004--Zhou-X-W--Fe--LAMMPS--ipr2,1.437,0.143,9.0
27,2010--Malerba-L--Fe--LAMMPS--ipr1,1.428,0.152,10.0
52,2018--Zhou-X-W--Fe-Ni-Cr--LAMMPS--ipr1,1.760,-0.180,11.0
53,2018--Zhou-X-W--Fe-Ni-Cr--LAMMPS--ipr2,1.760,-0.180,11.0
